In [1]:
# Uninstall packages to make sure most up to date are imported below
import sys
!{sys.executable} -m pip uninstall -y lusid-notifications-sdk-preview
!{sys.executable} -m pip uninstall -y lusid-workflows-sdk-generated-preview
!{sys.executable} -m pip uninstall -y finbourne-sdk-utilities

Found existing installation: lusid-notifications-sdk-preview 0.1.260
Uninstalling lusid-notifications-sdk-preview-0.1.260:
  Successfully uninstalled lusid-notifications-sdk-preview-0.1.260


## Workflows and Notifications Test

This notebook runs through a use case for the new workflows and notification endpoints. The use case is as follows:
* An issue occurs within LUSID
* An event in LUSID is created to report the issue (these are manually triggered for now)
* An email notification is triggered to report the event
* An API call is triggered by the event to create an entity that stores the details of the issue and represents it in LUSID
* A workflow is created to resolve the issue
* The workflow is transitioned through it's various states to resolve the issue (again events here are manually triggered for now)

Other limitations include not being able to group multiple issues to a single wokflow. This is because custom entities (used to represent issues) cannot be related.

In [2]:
# Import general purpose packages
import os
import json
from datetime import datetime, timedelta
import pytz

# Import lusid specific packages
import lusid
import lusid.models as models
from lusid.utilities import ApiClientFactory as lusid_factory
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.seed_sample_data import seed_data
from lusidtools.cocoon.utilities import create_scope_id

# Import notifications and workflows sdk preview packages
import sys
!{sys.executable} -m pip install lusid-notifications-sdk-preview
!{sys.executable} -m pip install lusid-workflows-sdk-preview
!{sys.executable} -m pip install finbourne-sdk-utilities
import lusid_notifications as ln
import lusid_workflows as lw
from fbnsdkutilities import ApiClientFactory
# Import data wrangling packages
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", None)

# Authenticate our user and create our API client
from lusidtools.cocoon import load_from_data_frame
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
lusid_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

lusid_api_url = lusid_factory.api_client.configuration.host
notifications_url = lusid_api_url[: lusid_api_url.rfind("/") + 1] + "notifications"
workflows_url = lusid_api_url[: lusid_api_url.rfind("/") + 1] + "workflows"

notifications_factory = ApiClientFactory(
    sdk=ln,
    token=RefreshingToken(),
    api_url=notifications_url
)

workflows_factory = ApiClientFactory(
    sdk=lw,
    token=RefreshingToken(),
    api_url=workflows_url
)

# Setup the apis we'll use in this notebook:
property_definitions_api = lusid_factory.build(lusid.api.PropertyDefinitionsApi)
custom_entity_definitions_api = lusid_factory.build(lusid.api.CustomEntityDefinitionsApi)
custom_entity_api = lusid_factory.build(lusid.api.CustomEntitiesApi)
subscriptions_api = notifications_factory.build(ln.api.SubscriptionsApi)
notifications_api = notifications_factory.build(ln.api.NotificationsApi)
events_api = notifications_factory.build(ln.api.EventsApi)
workflows_api = workflows_factory.build(lw.api.WorkflowsApi)

print ('LUSID Environment Initialised')
print ('LUSID SDK Version: ', lusid_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)

/opt/conda/lib/python3.9/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


  Using cached lusid_notifications_sdk_preview-0.1.260-py3-none-any.whl (93 kB)
  Using cached finbourne_sdk_utilities-0.0.7-py3-none-any.whl (16 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dve-lumipy-preview 0.1.150 requires decorator==4.3, but you have decorator 5.1.0 which is incompatible.
dve-lumipy-preview 0.1.150 requires networkx==2.5, but you have networkx 2.6.3 which is incompatible.
LUSID Environment Initialised
LUSID SDK Version:  0.6.8976.0


## 1. Notebook Setup

### 1.1 Create an identifier

The process in the notebook involves creating a custom entity to represent the issue flagged by a notification. When this is created it will need an identifier. This cell ensures an appropriate identifier is defined.

### 1.3 Create event subscriptions

These cells create subscriptions to the events that will be used in the notebook. From these subscriptions notifications can be set up to alert various users or automically kick off follow on actions. The events that will be used are as follows:
* New Issue
* Workflow Created
* Workflow Assigned
* Workflow In Progress
* Workflow Completed

In [3]:
# create subscriptions to our manual events used in this notebook

def create_event_subscriptions(subscription_scope, subscription_code, display_name, event_filter):
    event_subscription = ln.CreateSubscription(
                id= ln.ResourceId(
                    scope=subscription_scope,
                    code=subscription_code
                ),
                display_name=display_name,
                description="Subscription to a manual event",
                status="Active",
                matching_pattern= ln.MatchingPattern(
                    event_type="Manual",
                    filter="Message eq '"+event_filter+"'"
                )
    )
    
    # create susbcription definition
    try:
        create_subscription_event = subscriptions_api.create_subscription(
            create_subscription=event_subscription
        )

    except lusid.ApiException as e:
        print(json.loads(e.body))

    return event_subscription

In [6]:
# Setup the subscription details
event_filter = ['AllocationLoadCompleted']

subscription_scope = 'subscriptionscope'
    
# Pass event filters through subscription definition    
for element in event_filter:
    subscription_code = element
    display_name = element
        
    event_subscription = create_event_subscriptions(subscription_scope, subscription_code, display_name, element)

### 1.4 Create email notifications

This cell sets up email notifications for the events that have been subscribed to above. This means the user will recieve an email for each of the events triggered throughout this notebook.

In [7]:
# create email notifications for each of the subscriptions

scope="email_scope",
code="email_code1",
create_email_notification= ln.CreateEmailNotification(
            description="Email Event Notification",
            subject="Event with id of",
            plain_text_body="Event with message and details",
            email_address_to=["lorenz.declercq@finbourne.com"]
            )
        
for element in event_filter:
    email_notification_code = element
    email_notification_scope = 'subscriptionscope'
    
    try:
        create_email_notifications_response = notifications_api.create_email_notification(email_notification_scope, email_notification_code, create_email_notification
        )
        print(create_email_notifications_response)

    except lusid.ApiException as e:
        print(e)

{'content': {'HtmlBody': None,
             'PlainTextBody': 'Event with message and details',
             'Subject': 'Event with id of'},
 'created_at': datetime.datetime(2022, 4, 7, 13, 35, 17, 101199, tzinfo=tzlocal()),
 'created_by': '00uet0ll3vnLqW3SY2p7',
 'delivery_channel': 'Email',
 'description': 'Email Event Notification',
 'id': '428f9843-fb36-4420-bd55-8011d29b6331',
 'last_modified_at': datetime.datetime(2022, 4, 7, 13, 35, 17, 101199, tzinfo=tzlocal()),
 'last_modified_by': '00uet0ll3vnLqW3SY2p7',
 'recipients': {'EmailAddressBcc': [],
                'EmailAddressCc': [],
                'EmailAddressTo': ['lorenz.declercq@finbourne.com']},
 'status': {'last_updated': datetime.datetime(2022, 4, 7, 13, 35, 17, 101199, tzinfo=tzlocal()),
            'result': 'Unprocessed'}}


In [8]:
# create manual event for a new issue
body= dict(
    EventType = "Manual",
    Id = "abcdefghijk12345",
    Message = "AllocationLoadCompleted",
            subject="Event with id of",    
    Details ="Test details",
    EventTime ="2021-08-27T17:39:02.9427036+01:00"
)

try:
    create_event_response = events_api.create_event(body)
    print(create_event_response)
except lusid.ApiException as e:
    print(json.loads(e.body))
    
#     "acb5722d-341a-4802-b9fd-cf740a6a7797"

{}
